In [1]:
import requests
import os

In [2]:
OWNER = 'likr'
REPO = 'kplib'
BASE_PATH = '00Uncorrelated'
RAW_BASE = f'https://raw.githubusercontent.com/{OWNER}/{REPO}/master'

In [3]:
LOCAL_BASE_DIR = '../instances'  

In [4]:
# dohvatamo stranicu
def download_file(raw_url, local_name):
    os.makedirs(LOCAL_BASE_DIR, exist_ok=True)
    local_path = os.path.join(LOCAL_BASE_DIR, local_name)
    print(f'Downloading {raw_url} to {local_path}')
    r = requests.get(raw_url)
    r.raise_for_status()
    with open(local_path, 'wb') as f:
        f.write(r.content)

In [5]:
# "sredjuje" nazive fajlova
def sanitize_name(name):
    prefix = BASE_PATH + '/'
    if name.startswith(prefix):
        name = name[len(prefix):]
    return name.replace('/', '_').replace('\\', '_')

In [6]:
# dohvatamo instance
def process_path(path):
    api_url = f'https://api.github.com/repos/{OWNER}/{REPO}/contents/{path}'
    resp = requests.get(api_url)
    resp.raise_for_status()
    items = resp.json()

    for item in items:
        if item['type'] == 'file' and item['name'].endswith('.kp'):
            # napravi ime fajla sa putanjom da bude unikatno
            unique_name = sanitize_name(item['path'])
            raw_url = f"{RAW_BASE}/{item['path']}"
            download_file(raw_url, unique_name)
        elif item['type'] == 'dir':
            process_path(item['path']) 

In [7]:
process_path(BASE_PATH)